<h3>SENTIMENT ANALYSIS</h3>

Getting the final translated data

In [35]:
import re
train_tweets=[]
train_tags=[]
test_tweets=[]
test_tags=[]
d={'negative':0,'neutral':1,'positive':2}
with open('./help_files/train_final_trans.txt','r') as f:
    k = f.readlines()
    for i in k:
        l=i.strip().split()
        train_tweets.append(' '.join(l[1:-1]))
        train_tags.append(d[l[-1]])
with open('./help_files/test_final_trans.txt','r') as f:
    k = f.readlines()
    for i in k:
        l=i.strip().split()
        test_tweets.append(' '.join(l[1:-1]))
        test_tags.append(d[l[-1]])

In [2]:
import csv
mapping=['__label__NEGATIVE','__label__NEUTRAL','__label__POSITIVE']
def filter(tweets,tweet_tags):
    out_tweets=[]
    for i in range(len(tweets)):
        tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweets[i]).split())
        tweet = tweet.lower()
        if 'ø' in tweet or  'Ø' in tweet:
            tweet=tweet
        else:
            tweet = tweet.encode('ascii', 'ignore')
            tweet = tweet.decode("utf-8")
        out_tweets.append([mapping[tweet_tags[i]],str(tweet)])
    return(out_tweets)
train=filter(train_tweets,train_tags)
test=filter(test_tweets,test_tags)
with open('./help_files/train.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(train)
with open('./help_files/test.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(test)

        

So after translating to english and preprocessing text this is fasttext by facebook research for classification

In [23]:

import fasttext as fastText
#hyper_params= {"lr": 0.2,"epoch": 2,"wordNgrams": 2,"dim": 30,}     
hyper_params= {"lr": 0.4,"epoch": 2,"wordNgrams": 2,"dim": 18,}     

model = fastText.train_supervised(input='./help_files/actual_train.txt', **hyper_params)
test=test_tweets

This is for saving data and calculating metrics

In [36]:
l=[['tweet','actual','predicted']]
for i in range(len(test)):
    l.append([test[i],test_tags[i],model.predict(test[i])])
with open('./help_files/output.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(l)
import csv
y_true=[]
y_pred=[]
total=0
correct=0
wrong=0
d={'__label__NEUTRAL':1,'__label__NEGATIVE':0,'__label__POSITIVE':2}
c=0
with open('./help_files/output.csv','rt')as f:

    data = csv.reader(f)
    for row in data:
        c+=1

        if c>1:
            k=row[2]
            st=k.find('_')
            end=k.find(',')
            total+=1
            y_pred.append(int(d[row[2][st:end-1]]))
            y_true.append(int(row[1]))
            if int(d[row[2][st:end-1]])==int(row[1]):

                correct+=1
            else:
                wrong+=1




THE METRICS ARE AS FOLLOWS

In [34]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
prec=classification_report(y_true,y_pred)
print('accuracy is: ',acc)
print('\n')
print('*'*50)

print('classsification report')
print('\n')
print('\n')


print(prec)

accuracy is:  0.566916488222698


**************************************************
classsification report




              precision    recall  f1-score   support

           0       0.57      0.56      0.57       532
           1       0.54      0.54      0.54       754
           2       0.60      0.60      0.60       582

    accuracy                           0.57      1868
   macro avg       0.57      0.57      0.57      1868
weighted avg       0.57      0.57      0.57      1868

